In [18]:
import torch
import torch.nn as nn

In [24]:
class block(nn.Module):
    def __init__(self, in_channels, out_channels, expansion, identity_downsample=None, stride=1):
        super(block, self).__init__()
        self.expansion = expansion
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample # To reshape before feeding with pre activation of conv3
        
        
    def forward(self, x):
        identity = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
            
        x += identity
        x = self.relu(x)
        
        return x    
    

In [25]:
    
class ResNet(nn.Module):
    def __init__(self, block, layers, in_channels, num_classes):
        super(ResNet, self).__init__()
        
        #Initial convolutions and max pooling
        self.res_in_channels = 64
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        #ResNet layer definition
        self.layer1 = self.make_layer(block, layers[0], 64, 1)
        self.layer2 = self.make_layer(block, layers[1], 128, 2)
        self.layer3 = self.make_layer(block, layers[2], 256, 2)
        self.layer4 = self.make_layer(block, layers[3], 512, 2)
        
        #Average pooling to reduce each sample to 1*1*(512*4)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        
        #Linear layer
        self.fc = nn.Linear(512*4, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def make_layer(self, block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []
        expansion = 4
        
        if stride !=1 or self.res_in_channels != out_channels*4:
            identity_downsample = nn.Sequential(nn.Conv2d(self.res_in_channels, out_channels*expansion, kernel_size=1, stride=stride),
                                                nn.BatchNorm2d(out_channels*expansion))
        
        layers.append(block(self.res_in_channels, out_channels, expansion, identity_downsample, stride))
        self.res_in_channels = out_channels*expansion
        
        for i in range(num_residual_blocks-1):
            layers.append(block(self.res_in_channels, out_channels, expansion))
        
        return nn.Sequential(*layers)

In [26]:
def ResNet50(img_channels=3, num_classes=1000):
    return ResNet(block, [3, 4, 6, 3], img_channels, num_classes)

def ResNet101(img_channels=3, num_classes=1000):
    return ResNet(block, [3, 4, 23, 3], img_channels, num_classes)

def ResNet151(img_channels=3, num_classes=1000):
    return ResNet(block, [3, 8, 36, 3], img_channels, num_classes)
    

In [27]:
x = torch.randn(2, 3, 224, 224)
model = ResNet50()
y = model(x)
print(y.shape)    

torch.Size([2, 1000])
